# Proximal policy optimization

The following is the supplemental material for a paper I published in the journal Optica Quantum which can be found at this link. It is a short introduction to the proximal policy optimization algorithm for reinforcement learning that we used in the paper. 

## Brief Introduction to Reinforcement Learning with Actor-Critic Methods

Given an environment which can be modeled as a Markov decision process with state space $\mathcal{S}$, action space $\mathcal{A}$, state transition function $\mathcal{T}(s,a)$ and reward function $R(s)$ the goal of reinforcement learning is to learn an optimal policy $\pi^*(s)$ from experiences interacting with the environment. A policy defines how an agent selects an action when in a particular state $s_t$ at time step $t$ such that the sum of discounted future rewards, defined as 
\begin{align*}
\sum_{k=0}^{\infty}\gamma^kR_{t+k+1},
\end{align*}
is maximized. The hyperparameter $\gamma \in [0,1]$ is the discount factor and controls how much we discount rewards received at later time steps. The closer $\gamma$ is to 0 the greater rewards received at earlier time steps are weighted than those received at later time steps. In general polices are functions mapping states to probabilities of selecting each possible action. 
Another important function in reinforcement learning is the value function $V_\pi(s)$. Canonically the value function is defined as 
\begin{align*}
    V_\pi(s) = \mathbb{E}_\pi\left[\sum_{k=0}^{\infty}\gamma^kR_{t+k+1}\bigg| S_t=s\right].
\end{align*}
This is the expected sum of discounted future rewards when the agent begins in state $s$ at time step $t$ and follows the policy $\pi$ for the remaining time steps. The expectation $\mathbb{E}_{\pi}[\cdot]$ is used because both the policy and the environment are in general stochastic and the reward is thus a random variable. This function provides us a way to evaluate policies since a policy $\pi$ can be defined to be better than policy $\pi'$ if $V_{\pi}(s)\geq V_{\pi'}(s)$ for all $s\in \mathcal{S}$ {cite:p}`sutton2018reinforcement`.

For our reinforcement learning algorithm we use the $\texttt{StableBaselines3}$ implementation of the Proximal Policy Optimization (PPO) algorithm {cite:p}`schulman2017proximal`. $\texttt{StableBaselines3}$ is an open source reinforcement learning library that provides implementations of several common reinforcement learning algorithms {cite:p}`raffin2021stable`. PPO comes from a broader class of reinforcement learning methods called actor-critic methods {cite:p}`sutton2018reinforcement`. The actor is meant to learn an optimal policy while the critic evaluates the policy. 

The actor is modeled as a neural network and selects the optimal action given the state of the environment as it's input. In this work the state was represented as the flattened density matrix of the input quantum state $\rho_j$ i.e.
$s_j=[\textrm{Re}(\rho_j^{u}), \textrm{Im}(\rho_j^{u}), 
\textrm{diag}(\rho_j)] \in \mathbb{R}^{N^2}$ where $\rho_j^{u}$ denotes all entries of $\rho_j$ that are above its diagonal and $N$ is the dimension of the Hilbert space. 
The actor network has three fully connected hidden layers of size 256, 128, and 64 respectively. The output layer of the network consists of three outputs. 
These outputs are  $\mu_1$, $\mu_2$ and $\sigma$ where $\mu_1$ and $\mu_2$ are the means of two Gaussian distributions respectively and $\sigma$ is their standard deviation. 
The first action (the squeezing parameter $r$) is then sampled from the Gaussian $\mathcal{N}(\mu_1, \sigma)$ and the second action (the transmitivity of the beamsplitter $\tau$) is sampled from the Gaussian $\mathcal{N}(\mu_2, \sigma)$. 
At the start of training the standard deviation is set to 1 and typically decreases as the training progresses and the agent begins moving away from exploration. The policy is thus stochastic during training to allow for exploration. However, during inference (evaluation) the policy is made deterministic by always selecting the mean of the Gaussian as the action.
The critic is also modeled as a neural network with the same input and hidden layer architecture  with the only difference being that there is only one output. Given the state of the environment the critic attempts to approximate the value function $V_\pi(s)$ where $\pi$ is the current policy learned by the actor. We give an approximate outline of the $\texttt{StableBaselines3}$ implementation of PPO in Algorithm \ref{alg:ppo} and go over the details in the next section. Note that in lines 33 and 37 of the algorithm we use the stochastic gradient ascent update rule for updating the network parameters for brevity, however in the numerical experiments presented in the main text we use the Adam optimizer which has a more complex update rule {cite:p}`goodfellow2016deep`.

```{prf:algorithm} Pseudocode of $\texttt{StableBaselines3}$ implementation of PPO with deep neural networks
:label: sb3-ppo

1. Initialize empty rollout buffer $M=\{\}$ of maximum capacity  $N$  Comment{$N$ is $\texttt{n_steps}$ from Table I of the main material}
2. Initialize policy (actor) network $\pi$ with random weights $\theta$
3. Initialize value (critic) network $\hat{V}$ with random weights $\phi$
4. Set number of epochs $n$ to optimize over Comment{$n$ is $\texttt{n_epochs}$ from Table I of the main material}
5. Set discount factor $\gamma$ Comment{$\gamma$ is $\texttt{gamma}$ from Table I of the main material}
6. $i \gets 0$
7. While $i < T$: Comment{$T$ is the maximum number of times the agent can interact with the environment during training}
    1. Initialize array $E$ of $K$ environments all in initial state $s_t=s_0$
    2. $j \gets 0$
    3. While $j < N$:
        1. For $k=0$ to $K$: Comment{This loop can be parallelized since each environment is independent of the others}
            1. ($\mu_1,\mu_2, \sigma) \gets \pi(s_t)$ Comment{Get means and standard deviation of the two action distributions from $\pi$}
            2. $a_1 \sim \mathcal{N}(\mu_1,\sigma)$ Comment{Sample action 1 from distribution}
            3. $a_2 \sim \mathcal{N}(\mu_2,\sigma)$
            4. $a_t \gets (a_1, a_2)$
            5. $(s_{t+1}, r_t) \gets E[k](s_t, a_t, \gamma)$ Comment{Perform action $a_t$ and get state transition and reward}
            6. Store experience $(s_t,a_t,r_t,s_{t+1})$ in $M$
            7. $s_t \gets s_{t+1}$ \Comment{Update state of current ($k$-th) environment}
            8. $i \gets i+1$
            9. $j \gets j+1$

    4. Compute advantages $A_t$ for experiences in $M$ using predictions from $\hat{V}$ 
    5. For $e=0$ to $n$:  Comment{Policy improvement step}
       1. Divide $M$ into batches of size $b$ and store them in $B$ Comment{$b$ is $\texttt{batch_size}$ from Table I of the main material}
       2. For batch in $B$:
            1. Compute gradient estimate $\hat{g}_\theta$ which is the average of $\nabla_\theta L_t^{CLIP+ENT}(\theta)$ over batch
            2. Clip norm of $\hat{g}_\theta$ to be $< |G|_{max}$ Comment{$|G|_{max}$ is $\texttt{max_grad_norm}$ from Table I of the main material}
            3. $\theta \gets \theta + \alpha\hat{g}_\theta$ 
            4. Compute gradient estimate $\hat{g}_\phi$ which is the average of $\nabla_\phi L_t^{VF}(\phi)$ over batch
            5. Clip norm of $\hat{g}_\phi$ to be $< |G|_{max}$
            6. $\phi \gets \phi + \alpha\hat{g}_\phi$ Comment{$\alpha$ is $\texttt{learning_rate}$ from Table I of the main material}

    4. Reinitialize $M=\{\}$ to empty rollout buffer of maximum capacity $N$ 
```

## Brief Introduction to PPO and Policy Gradient Methods

Many earlier breakthroughs of reinforcement learning, particularly for game playing, utilized action-value methods like $Q$-learning {cite:p}`mnih2015human`. However, these methods are not easily applicable to environments where the action space is very large or continuous [{cite:p}`sutton2018reinforcement`, {cite:p}`schulman2017proximal`]. In this setting it is more advantageous to utilize another set of reinforcement learning methods known as policy gradient methods of which PPO is an example 

```{note}
:class: dropdown

PPO is also what is known as an on-policy method meaning the policy used to gather experiences for policy improvement (training) is the same policy that is updated during policy improvement. Off-policy methods like Deep $Q$-learning utilize a separate policy to gather experiences for policy improvement such as an $\epsilon$-greedy policy. Note that one disadvantage of PPO being an on-policy algorithm is that it is not as sample efficient as off-policy methods such as Deep $Q$-learning or Soft Actor Critic, as seen in line 41 of Algorithm \ref{alg:ppo}, so for environments that are expensive to sample from the advantage of PPO is less clear.

```

These methods seeks to learn a parameterized policy $\pi_{\theta}$ directly which selects actions without the need to consult a value function like action-value methods such as $Q$-learning {cite:p}`sutton2018reinforcement`. Note, however, that although a value function is not needed for action selection it may still be used to learn the policy parameters as is the case with actor-critic methods.

Given a policy $\pi_{\theta}$ parameterized by $\theta$ vanilla policy gradient methods attempt to optimize the policy by maximizing the objective function 
\begin{align*}
    L_t^{VPG}(\theta) = \mathbb{\hat{E}}_t\left[\log{(\pi_\theta(a_t|s_t))} \hat{A}_t \right]
\end{align*}
using some gradient ascent algorithm.
This is done by computing an estimate of the gradient of the objective function (the policy gradient) with respect to the policy parameters $\theta$. The  gradient estimator is of the form
\begin{equation*}
    \hat{g} = \mathbb{\hat{E}}_t\left[ \nabla_\theta\log{(\pi_\theta(a_t|s_t))} \hat{A}_t \right]
\end{equation*}
where $\pi_\theta(a|s)$ is the probability of taking action $a$ given the state $s$ and $\hat{A}_t$ is an estimator of the advantage function at time step $t$. The advantage function is given by 
\begin{align*}
    A_t = \left[\sum^{t_{end}}_{k=t}\gamma^{k-t}R_{k}\right]-V(s_t)
\end{align*}
where $t_{end}$ is the final time step. The first term is the discounted sum of rewards computed for an episode starting from time step $t$. The second term is the value function introduced previously. When performing the optimization we do not have the function $V(s_t)$ so we use an estimator for it $\hat{V}(s_t)$ which is modeled by the critic network so the estimator of the advantage function is computed as 
\begin{align*}
    \hat{A}_t = \left[\sum^{t_{end}}_{k=t}\gamma^{k-t}R_{k}\right]-\hat{V}(s_t).
\end{align*}
If the advantage function is positive, i.e. the discounted sum of rewards for the episode was greater than what the critic predicted it would be, we would want to increase the probability of selecting action $a_t$ given the current state $s_t$ in future episodes  and decrease the probability if the advantage is negative.

PPO has one more crucial component that it borrows from trust region policy  methods such as trust region policy optimization (TRPO). That is it constrains how much the policy can change during each policy update. This is needed because a policy update can result in the policy (network) parameters changing so greatly that the network will ``overfit'' to the current batch of experiences being used resulting in a catastrophic collapse in performance for unseen states. TRPO attempts to solve this by optimizing the objective function, also called the surrogate objective, 
\begin{align*}
    L_t^{TRPO}(\theta)=\mathbb{\hat{E}}_t\left[\frac{\pi_\theta(a_t|s_t)}{{\pi_{\theta}}_{\textrm{old}}(a_t|s_t)}\hat{A}_t\right]
\end{align*}
with respect to the parameters $\theta$ where $\theta_{\textrm{old}}$ are the parameters of the policy before the policy update. Additionally this surrogate function is maximized with the constraint 

$$
\begin{align*}
    \mathbb{\hat{E}}_t\left[ \textrm{KL}[\pi_{\theta_{\textrm{old}}}(\cdot|s_t), \pi_{\theta}(\cdot|s_t)] \right] \leq \delta
\end{align*}
$$
 

where $\textrm{KL}[\cdot,\cdot]$  is the Kullback–Leibler divergence between the new policy and the old policy and $\delta$ is some constant. Using $L_t^{TRPO}(\theta)$ as the objective function subject to the given $\textrm{KL}$ constraint however requires second order optimization via computation of the Hessian which is more expensive than just computing the gradient. This was in part what motivated the creation of PPO as it does not require computation of the Hessian.

In the case of PPO the algorithm attempts to maximize the objective function
\begin{equation*}
   L_t^{PPO}(\theta)= L_t^{CLIP+VF+ENT}(\theta)=\mathbb{\hat{E}}_t\left[L_t^{CLIP}(\theta)-c_1L_t^{VF}(\theta) + L^{ENT}_t(\theta) \right].
\end{equation*}
The first term is expressed as
\begin{equation*}
    L_t^{CLIP}(\theta)=\mathbb{\hat{E}}_t\left[\min(r_t(\theta)\hat{A}_t,\textrm{clip}(r_t(\theta),1-\epsilon,1+\epsilon)\hat{A}_t)\right]
\end{equation*}
where $r_t(\theta)=\frac{\pi_\theta(a_t|s_t)}{{\pi_{\theta}}_{\textrm{old}}(a_t|s_t)}$ and $\epsilon$ is a hyperparameter (called $\texttt{clip_range}$ in Table I of the main material) controlling how far the new policy may deviate from the old policy {cite:p}`schulman2017proximal`. This term serves the same purpose $L^{TRPO}(\theta)$ does in TRPO i.e. to constrain the magnitude of policy updates. 
The second term is expressed as
\begin{equation*}
    L_t^{VF}=(\hat{V}(s_t)-V_t^{actual})^2=\left(\hat{V}(s_t)-\sum^{t_{end}}_{k=t}\gamma^{k-t}R_{k}\right)^2
\end{equation*}
and is the squared difference between the sum of discounted rewards predicted by the value network and the actual sum of discounted rewards received after time step $t$.
The coefficient $c_1$ (called $\texttt{vf_coef}$ in Table I of the main material) is used to increase or decrease the contribution of this term to the over all objective function $L_t^{PPO}(\theta)$. The last term in the objective function 
is expressed as
\begin{align*}
    L^{ENT}_t(\theta) = c_2S[\pi_{\theta}](s_t)
\end{align*} 
and is an entropy bonus computed using the entropy of the probability distributions used to select the actions [{cite:p}`bishop2006pattern`, {cite:p}`bertsekas2008introduction`]. This term is used to encourage exploration if the stochastic nature of action selection during training provides insufficient exploration of the environment. The coefficient $c_2$ (called $\texttt{ent_coef}$ in Table I of the main material) serves the same purpose as $c_1$ in the previous term. Note that while the combined objective function $L_t^{PPO}(\theta)$ includes all three terms the $\texttt{StableBaselines3}$ implementation of PPO uses a separate network for the actor and critic by default, as opposed to sharing parameters between the two networks as it did in version $\texttt{v1.8.0}$ and below, and thus only the second term of the objective function is used to optimize the critic network while the first and third terms are used to optimize the actor network.